In [76]:
# Sam Brown
# sam_brown@mines.edu
# June 27 2025
# Goal: Use a regular neural network with our new features (tidal amplitudes between events) to predict slip size

import sys
sys.path.append("/Users/sambrown04/Documents/SURF/whillans-surf/notebooks/SURF")

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

df = pd.read_csv("/Users/sambrown04/Documents/SURF/Preproc_data/upd10-18.csv")
df = df[604:3001] # Adjust for missing data areas if needed

In [78]:
# Will do a little bit of preprocessing to see how including more in depth H,L 
sequence = [1 if x == 1 else -1 for x in df['high_t_evt']]

df['cumult'] = np.cumsum(sequence)
df['hpastone'] = df['high_t_evt'].shift(1)
df['hpasttwo'] = df['high_t_evt'].shift(2)
df['hpastthree'] = df['high_t_evt'].shift(3)

df =  df[610:3001]

In [176]:
# Features and target
X = df[['tide_deriv', 'time_since', 'high_t_evt', 'tide_height', 'A_diurn', 'A_semidiurn', 'sev_stds', 'cumult', 'hpastone', 'disp_standardized', 'r2_standardized', 'slope_standardized']]
y =df['slip_size_standardized'].values.reshape(-1,1)
 
# Split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2, random_state = 42)

# Standardize 
X_scaler = StandardScaler()
X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)

In [178]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(12, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64,32)
        self.output = nn.Linear(32,1)
        
    def forward(self,x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.output(x) # regressing so no activation
        return x

In [180]:
model = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = .01)

In [182]:
# Train
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad() # Clears grad

    # predictions and loss
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    #Backprop
    loss.backward()

    #Update
    optimizer.step()

    if (epoch+1) % 20 == 0: # Print update to ensure no problems
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [20/200], Loss: 0.1201
Epoch [40/200], Loss: 0.0750
Epoch [60/200], Loss: 0.0561
Epoch [80/200], Loss: 0.0422
Epoch [100/200], Loss: 0.0344
Epoch [120/200], Loss: 0.0330
Epoch [140/200], Loss: 0.0265
Epoch [160/200], Loss: 0.0234
Epoch [180/200], Loss: 0.0244
Epoch [200/200], Loss: 0.0205


In [184]:
model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor)
    y_pred = y_scaler.inverse_transform(y_pred_scaled.detach().numpy())
    y_test_orig = y_scaler.inverse_transform(y_test_tensor.detach().numpy())
r2 = r2_score(y_test_orig, y_pred)
print(r2)


0.9434876441955566
